## Sequence Classification with LSTMs

Tutorial from link: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/


In [1]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
numpy.random.seed(7)

/Users/etang/anaconda3/envs/nlu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
top_words = 5000
INDEX_FROM=3
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words, index_from=INDEX_FROM)
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.5)

In [3]:
print(X_train.shape)
print(X_cv.shape)
print(X_test.shape)

(25000,)
(12500,)
(12500,)


In [4]:
word_index = imdb.get_word_index()

In [5]:
word_index = {k: (v + INDEX_FROM) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["giraffe"]

48151

In [6]:
freq_index = {freq: word for word, freq in word_index.items()}
freq_index[48151]

'giraffe'

In [7]:
test_sent = [freq_index[i] for i in X_train[1]]
print(test_sent)

['<START>', 'big', 'hair', 'big', '<UNK>', 'bad', 'music', 'and', 'a', 'giant', 'safety', '<UNK>', 'these', 'are', 'the', 'words', 'to', 'best', 'describe', 'this', 'terrible', 'movie', 'i', 'love', 'cheesy', 'horror', 'movies', 'and', "i've", 'seen', 'hundreds', 'but', 'this', 'had', 'got', 'to', 'be', 'on', 'of', 'the', 'worst', 'ever', 'made', 'the', 'plot', 'is', 'paper', 'thin', 'and', 'ridiculous', 'the', 'acting', 'is', 'an', '<UNK>', 'the', 'script', 'is', 'completely', 'laughable', 'the', 'best', 'is', 'the', 'end', 'showdown', 'with', 'the', 'cop', 'and', 'how', 'he', 'worked', 'out', 'who', 'the', 'killer', 'is', "it's", 'just', 'so', 'damn', 'terribly', 'written', 'the', 'clothes', 'are', '<UNK>', 'and', 'funny', 'in', 'equal', '<UNK>', 'the', 'hair', 'is', 'big', 'lots', 'of', '<UNK>', '<UNK>', 'men', 'wear', 'those', 'cut', '<UNK>', '<UNK>', 'that', 'show', 'off', 'their', '<UNK>', '<UNK>', 'that', 'men', 'actually', 'wore', 'them', 'and', 'the', 'music', 'is', 'just', '<

In [8]:
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
X_cv = sequence.pad_sequences(X_cv, maxlen=max_review_length)

In [9]:
X_train.shape

(25000, 500)

In [10]:
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
model.fit(X_train, y_train, validation_data=(X_cv, y_cv), epochs=3, batch_size=64)

Train on 25000 samples, validate on 12500 samples
Epoch 1/3
25000/25000 [==============================] - 219s 9ms/step - loss: 0.4598 - acc: 0.7737 - val_loss: 0.4206 - val_acc: 0.8150
Epoch 2/3
25000/25000 [==============================] - 219s 9ms/step - loss: 0.3794 - acc: 0.8277 - val_loss: 0.3167 - val_acc: 0.8671
Epoch 3/3
25000/25000 [==============================] - 225s 9ms/step - loss: 0.2596 - acc: 0.8980 - val_loss: 0.3158 - val_acc: 0.8749


In [12]:
scores = model.evaluate(X_test, y_test, verbose=1)

12500/12500 [==============================] - 34s 3ms/step


In [13]:
print(model.metrics_names)
print("Accuracy: %.2f%%" % (scores[1]*100))

['loss', 'acc']
Accuracy: 87.44%


In [18]:
test_sent_index = 10
print(' '.join([freq_index[i] for i in X_test[test_sent_index]]))
print("Scored as: ", model.predict(X_test[test_sent_index:test_sent_index + 1]))

<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <START> it stars war <UNK> william <UNK> <UNK> who falls in love with a stunning <UNK> doctor 

In [35]:
test_sent = "When Day of the Soldado truly wallows in violence, it does so exquisitely, with the kind of hopelessness that film violence, especially around this subject matter, should convey"
test_vec = numpy.array([[(word_index[w] if (w in word_index and word_index[w] < top_words) else 0) for w in test_sent.lower().split()]])
print(test_vec)
print(word_index["deepen"])
test_vec = sequence.pad_sequences(test_vec, maxlen=max_review_length)
print("Scored as: ", model.predict(test_vec)[0])

[[  54  251    7    4    0  371    0   11    0   12  127   38    0   19
     4  243    7    0   15   22    0  262  187   14  875    0  144 2833]]
72704
Scored as:  [0.7371806]
